In [1]:
import datetime
import json
import requests

import pandas as pd

from requests_oauthlib import OAuth1Session
from urllib import parse

In [2]:
# Define key and token
dict_key = {}

with open('../../key-and-tokes.txt', 'r') as f:
    for line in f.readlines():
        pair = line.replace('\n', '').split(':')
        dict_key[pair[0].strip()] = pair[1].strip()

## 試しに自分のアカウントのtweetを取得

In [ ]:
# Set url
url = 'https://api.twitter.com/1.1/statuses/user_timeline.json'

# Set parameters
params = {'screen_name':'gan_informatics',
          'exclude_replies' : True,
          'include_rts' : False,
          'count' : 200}

In [ ]:
# API authentication
twitter = OAuth1Session(dict_key['Consumer Key'], dict_key['Consumer Secret'],
                        dict_key['Access Token'], dict_key['Access Token Secret'])

In [ ]:
# Request
res = twitter.get(url, params=params)

In [ ]:
# Get tweets
with open('../../data/twitter/sample_tweets.txt', 'w', encoding='UTF-8') as f:
    for j in range(100):
        res = twitter.get(url, params=params)

        if res.status_code == 200:

            # API残り
            limit = res.headers['x-rate-limit-remaining']
            print ("API remain: " + limit)
            if limit == 1:
                sleep(60*15)

            n = 0
            timeline = json.loads(res.text)
            # 各ツイートの本文を表示
            for i in range(len(timeline)):
                if i != len(timeline)-1:
                    f.write(timeline[i]['text'] + '\n')
                else:
                    f.write(timeline[i]['text'] + '\n')
                    #一番最後のツイートIDをパラメータmax_idに追加
                    params['max_id'] = timeline[i]['id']-1

## 特定の単語を含むtweetを検索

In [ ]:
search_query = '"川中島" "幻舞"'

parse.quote(search_query)

In [3]:
# Set url
url = 'https://api.twitter.com/1.1/search/tweets.json'

search_query = '"川中島" "幻舞"'

# Set parameters
params = {'q': parse.quote(search_query),
          'exclude_replies' : True,
          'include_rts' : False,
          'count' : 200}

max_ite = 10

txt_output = '../../data/twitter/kawanakajima_genbu.txt'

In [4]:
# API authentication
twitter = OAuth1Session(dict_key['Consumer Key'], dict_key['Consumer Secret'],
                        dict_key['Access Token'], dict_key['Access Token Secret'])

In [5]:
params

{'count': 200,
 'exclude_replies': True,
 'include_rts': False,
 'q': '%22%E5%B7%9D%E4%B8%AD%E5%B3%B6%22%20%22%E5%B9%BB%E8%88%9E%22'}

In [6]:
res = twitter.get(url, params=params)
tweets = json.loads(res.text)['statuses']

tweets

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Fri Dec 01 09:14:49 +0000 2017',
  'entities': {'hashtags': [],
   'symbols': [],
   'urls': [{'display_url': 'hashimotoyasaketen.cocolog-nifty.com/blog/2017/12/p…',
     'expanded_url': 'http://hashimotoyasaketen.cocolog-nifty.com/blog/2017/12/post-7124.html',
     'indices': [26, 49],
     'url': 'https://t.co/cAfNlpfK4k'}],
   'user_mentions': []},
  'favorite_count': 0,
  'favorited': False,
  'geo': None,
  'id': 936523971876438016,
  'id_str': '936523971876438016',
  'in_reply_to_screen_name': None,
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'is_quote_status': False,
  'lang': 'ja',
  'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'},
  'place': None,
  'possibly_sensitive': False,
  'retweet_count': 0,
  'retweeted': False,
  'source': '<a href="http://www.cocolog-nifty.com/" rel="nofollow">ココログフリー</a>',
  '

In [8]:
params['max_id'] = 897583014921650176 - 1
params

{'count': 200,
 'exclude_replies': True,
 'include_rts': False,
 'max_id': 897583014921650175,
 'q': '%22%E5%B7%9D%E4%B8%AD%E5%B3%B6%22%20%22%E5%B9%BB%E8%88%9E%22'}

In [9]:
res = twitter.get(url, params=params)
tweets = json.loads(res.text)['statuses']

tweets

[]

In [ ]:
df_tweets = pd.DataFrame(columns=['id', 'favorite_count', 'retweet_count', 'text'])

for ite in range(max_ite):
    # Request
    res = twitter.get(url, params=params)
    
    #Confirm response
    if res.status_code == 200:
        # Output tweet
        tweets = json.loads(res.text)['statuses']
        for tweet in tweets:
            df_tweets = df_tweets.append(pd.DataFrame(
                {'id': tweet['id'], 'favorite_count': tweet['favorite_count'],
                 'retweet_count': tweet['retweet_count'], 'text': tweet['text'].replace('\n', u'。')},
            index=[0]), ignore_index=True)
        
        # Print remaining API
        remaining_api = int(res.headers['x-rate-limit-remaining'])
        print('Iteration: %s/%s, Remaining API: %s'%(ite+1, max_ite, remaining_api))
        
        # Sleep 15 minutes if there is no remain
        if remaining_api <= 1:
            print('There is no remaining API.\nSleeping until %s'%(datetime.datetime.now() + datetime.timedelta(minutes=15)))
            
# Output df_tweets
df_tweets.loc[:, ['id', 'favorite_count', 'retweet_count', 'text']].to_csv(txt_output, sep='\t', index=False, encoding='utf-8')

In [ ]:
json.loads(res.text)

In [ ]:
json.loads(res.text)['search_metadata']

In [ ]:
len(json.loads(res.text)['statuses'])

In [ ]:
json.loads(res.text)['statuses'][0]['id']

In [ ]:
json.loads(res.text)['statuses'][-1]['id']

In [ ]:
print('There is no remaining API.\nSleeping until %s'%(datetime.datetime.now() + datetime.timedelta(minutes=15)))

In [ ]:
# Set text file for output
pd.DataFrame(columns=['id', 'favorite_count', 'retweet_count', 'text']).to_csv(sep='\t', index=False)

# Get tweets
with open('../../data/twitter/kawanakajima_genbu.txt', 'w', encoding='UTF-8') as f:
    for j in range(100):
        # Request
        res = twitter.get(url, params=params)

        if res.status_code == 200:

            # Remaining API
            limit = res.headers['x-rate-limit-remaining']
            print('API remain: %s'%(limit))
            if limit == 1:
                # Sleep 15 minutes
                sleep(60*15)

            n = 0
            timeline = json.loads(res.text)['statuses']
            
            # Write text
            for i in range(len(timeline)):
                f.write(timeline[i]['text'] + '\n')
                    
            # Add max_id as a parameter
            params['max_id'] = json.loads(res.text)['search_metadata']['max_id']

In [ ]:
res.status_code

In [ ]:
res = twitter.get(url, params=params)

timeline = json.loads(res.text)['statuses']

In [ ]:
timeline[0]['text'].replace('\n', u'。')

In [ ]:
timeline[0]['favorite_count']

In [ ]:
timeline[0]['retweet_count']

In [ ]:
timeline[0]['id']